# The University of Hong Kong
## DASC7600 Data Science Project 2024
## Compartmental Model

# Import Modules and Settings

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings
from scipy.signal import find_peaks, peak_widths

import covid_module
from compartmental_model import *

# Settings
warnings.filterwarnings('ignore')

# Load Data

In [2]:
# Read csv file
covid_hk_case_std = pd.read_csv('./data/std_data/hk/covid_hk_case_std.csv')

# New Case Counts

In [3]:
covid_hk_new_case_cnt_df = covid_module.get_date_count(covid_hk_case_std, 'report_date', '%Y%m%d')

# Fitting Model - First Wave

In [4]:
# New case counts for 1st wave
first_wave_new_case_cnt = covid_hk_new_case_cnt_df["count"].values[43:89].tolist()

In [5]:
# SIR_model_first_wave_hk = SIR_model(first_wave_new_case_cnt, [sum(first_wave_new_case_cnt), 1, 0], (0.001 ,1))
# plot_fitted_result(SIR_model_first_wave_hk.fit(), first_wave_new_case_cnt, SIR_model_first_wave_hk.classes)

In [6]:
# SIS_model_first_wave_hk = SIS_model(first_wave_new_case_cnt, [sum(first_wave_new_case_cnt), 1], (0.001 ,1))
# plot_fitted_result(SIS_model_first_wave_hk.fit(), first_wave_new_case_cnt, SIS_model_first_wave_hk.classes)

In [7]:
# SIRS_model_first_wave_hk = SIRS_model(first_wave_new_case_cnt, [sum(first_wave_new_case_cnt), 1, 0], (0.001 , 1, 0.1))
# plot_fitted_result(SIRS_model_first_wave_hk.fit(), first_wave_new_case_cnt, SIRS_model_first_wave_hk.classes)

In [8]:
# SEIR_model_first_wave_hk = SEIR_model(first_wave_new_case_cnt, [sum(first_wave_new_case_cnt), 1, 0, 0], (0.001 , 1, 0.1))
# plot_fitted_result(SEIR_model_first_wave_hk.fit(), first_wave_new_case_cnt, SEIR_model_first_wave_hk.classes)

# Wave Detection

In [9]:
new_case_cnt = covid_hk_new_case_cnt_df['count']
wave_peaks, _ = find_peaks(new_case_cnt, height=0, width=5) # Return: peaks (ndarray), properties (dict)
wave_widths = peak_widths(new_case_cnt, wave_peaks, rel_height=1) # Return: widths (ndarray), width_heights (ndarray), left_ips (ndarray), right_ips (ndarray)

In [10]:
# # Plot the new case counts with different waves identified
# plt.subplots(figsize=(15, 6))
# plt.plot(new_case_cnt)
# plt.plot(np.zeros_like(new_case_cnt), '--', color='gray')
# ## Peak of each wave
# plt.plot(wave_peaks, new_case_cnt[wave_peaks], 'x', color='green')
# ## Period of each wave
# plt.plot(wave_widths[2], wave_widths[1], '|', color='green')
# plt.plot(wave_widths[3], wave_widths[1], '|', color='green')
# plt.hlines(*wave_widths[1:], color='green')
# plt.title('Number of Covid-19 cases and waves identified (Hong Kong)')
# plt.show()

# Fitting Model - SIR on Detected Waves

In [11]:
nbr_wave = len(wave_widths[0])

wave_index_all = []
wave_predicted_cnt_all = []
for i in range(nbr_wave):
    wave_start_index = int(wave_widths[2][i])+1
    wave_end_index = int(wave_widths[3][i])+1
    wave_new_case_cnt = covid_hk_new_case_cnt_df["count"].values[wave_start_index:wave_end_index].tolist()
    SIR_model_hk = SIR_model(wave_new_case_cnt, [sum(wave_new_case_cnt), 1, 0], (0.001 ,1))
    wave_predicted_cnt = SIR_model_hk.fit()[SIR_model_hk.I_pos]
    wave_index_all.extend(range(wave_start_index, wave_end_index))
    wave_predicted_cnt_all.extend(wave_predicted_cnt)

In [12]:
# # Plot the new case counts with different waves identified and their predicted counts
# plt.subplots(figsize=(15, 6))
# plt.plot(new_case_cnt)
# plt.plot(np.zeros_like(new_case_cnt), '--', color='gray')
# ## Peak of each wave
# plt.plot(wave_peaks, new_case_cnt[wave_peaks], 'x', color='green')
# ## Period of each wave
# plt.plot(wave_widths[2], wave_widths[1], '|', color='green')
# plt.plot(wave_widths[3], wave_widths[1], '|', color='green')
# plt.hlines(*wave_widths[1:], color='green')
# plt.title('Number of Covid-19 cases and waves identified (Hong Kong)')
# # Predicted new case counts counts
# plt.plot(wave_index_all, wave_predicted_cnt_all, color='red')
# plt.show()